# Pruebo Stacking con los datos filtrados#

El stack consiste de los siguientes modelos como 1º paso:

    RandomForestClassifier

    ExtraTreesClassifier

    AdaBoostClassifier

    GradientBoostingClassifier
    
    XGBClassifier
    
Y como 2º paso MLPClassifier

Uso Random under sampling


In [149]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

import xgboost as xgb

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import BaggingClassifier
from sklearn.kernel_approximation import RBFSampler

### Cargo los Features###

In [150]:
labels_f = pd.read_csv('../../data/labels_f_filtrado.csv')
datos = pd.read_csv('../../data/datos_filtrado.csv')

In [151]:
atributos = labels_f.columns.tolist()

In [152]:
atributos.remove('person')
atributos.remove('label')

In [153]:
y = labels_f['label'].ravel()
X = labels_f.loc[:,atributos]

In [154]:
scaler = MaxAbsScaler().fit(X)
X = scaler.transform(X)

In [155]:
rbf_feature = RBFSampler(gamma=2,n_components= 300, random_state=123)
X = rbf_feature.fit_transform(X)



In [156]:
def aplicarSmote(X_train, y_train):
    smote = SMOTE(ratio='minority')
    X_smt_train, y_smt_train = smote.fit_sample(X_train, y_train)
    return (X_smt_train, y_smt_train)

In [157]:
def aplicarRus(X_train_p, y_train_p):
    rus= RandomUnderSampler(return_indices=True)
    #id_rus son los índices
    X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train_p, y_train_p)
    return(X_rus_train, y_rus_train)

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123, stratify = y)

In [159]:
X_train, y_train = aplicarRus(X_train,y_train)

In [160]:
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    


In [161]:
"""    
    for i,(train_index, test_index) in enumerate(kf.split(X_transformed)):
        x_train, x_test = X_transformed[train_index], X_transformed[test_index]
        y_train, y_test = y[train_index], y[test_index]
        try:
            x_tr = x_train[train_index]
            y_tr = y_train[train_index]
            x_te = x_train[test_index]
            
"""

'    \n    for i,(train_index, test_index) in enumerate(kf.split(X_transformed)):\n        x_train, x_test = X_transformed[train_index], X_transformed[test_index]\n        y_train, y_test = y[train_index], y[test_index]\n        try:\n            x_tr = x_train[train_index]\n            y_tr = y_train[train_index]\n            x_te = x_train[test_index]\n            \n'

In [162]:
def get_oof(clf, x_train_p, y_train_p, x_test_p):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    kf = KFold(n_splits= NFOLDS, random_state=SEED)
    kf = kf.split(x_train_p)
    i = 0

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train_p[train_index]
        y_tr = y_train_p[train_index]
        x_te = x_train_p[test_index]
        clf.fit(x_tr, y_tr)
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test_p)
        i+=1
        #print(i)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [163]:
rf_params = {'bootstrap': True, 'class_weight': None, 'criterion': 'gini',
            'max_depth': 9, 'max_features': 'auto', 'max_leaf_nodes': None,
            'min_impurity_decrease': 0.0, 'min_impurity_split': None,
            'min_samples_leaf': 1, 'min_samples_split': 2,
            'min_weight_fraction_leaf': 0.0, 'n_estimators': 1444, 'n_jobs': None,
            'oob_score': False, 'verbose': 0,
            'warm_start': False
}#'n_jobs': -1,

# Extra Trees Parameters
et_params = {'bootstrap': False, 'class_weight': None, 'criterion': 'gini',
          'max_depth': 100, 'max_features': 'auto','max_leaf_nodes': None,
           'min_impurity_decrease': 0.0, 'min_impurity_split': None,
           'min_samples_leaf': 5, 'min_samples_split': 5,
           'min_weight_fraction_leaf': 0.0, 'n_estimators': 200,'n_jobs': 4}

# AdaBoost parameters
ada_params = {'algorithm': 'SAMME.R', 'base_estimator': None,
          'learning_rate': 0.01, 'n_estimators': 977, 'random_state': None}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

xgb_params = {'colsample_bylevel': 0.8805332918491438, 'colsample_bytree': 0.9352504152264935, 'gamma': 9.881437744998433e-06, 'learning_rate': 0.19933863096197899, 'max_delta_step': 6, 'max_depth': 18, 'min_child_weight': 3, 'n_estimators': 61, 'reg_alpha': 7.871401820904661e-05, 'reg_lambda': 1000.0, 'scale_pos_weight': 5.390140010639928, 'subsample': 1.0}

In [164]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
xgb =  SklearnHelper(clf=XGBClassifier, seed=SEED, params=xgb_params)
#svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
#nn = SklearnHelper(clf = MLPClassifier, seed = SEED, params = nn_params)

In [165]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, X_train, y_train, X_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,X_train, y_train, X_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, X_train, y_train, X_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,X_train, y_train, X_test) # Gradient Boost
xgb_oof_train,xgb_oof_test = get_oof(xgb,X_train, y_train, X_test) # Support Vector Classifier
#nn_oof_train, nn_oof_test = get_oof(nn,X_train, y_train, X_test)
print("Training is complete")

Training is complete


In [166]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train,xgb_oof_train ), axis=1) #
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test,xgb_oof_test), axis=1)

In [167]:
nn = MLPClassifier( early_stopping=False, random_state=123).fit(x_train, y_train)#, early_stopping_rounds = 5, eval_set=[(x_test, y_test)])
#bag_class =BaggingClassifier(nn).fit(x_train, y_train)

In [168]:
preds = nn.predict(x_test)
preds_prob = nn.predict_proba(x_test)[:,1]
train_accuracy = accuracy_score(y_train, bag_class.predict(x_train))
test_accuracy = accuracy_score(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
matriz_de_confusion = confusion_matrix(y_test, preds)
print('train acurracy: ')
print(train_accuracy)
print('test acurracy: ')
print(test_accuracy)
print('Matriz de confusión: ')
print(matriz_de_confusion)
print('Área bajo la curva: ')
print(area_debajo_de_curva)

train acurracy: 
0.6649659863945578
test acurracy: 
0.586508753861998
Matriz de confusión: 
[[1054  790]
 [  13   85]]
Área bajo la curva: 
0.7585937845854177
